In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=297140119d38d6c8031eaf2ea38fc89c030a2e562e6c2d9f89d76d1077191827
  Stored in directory: /root/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [ ]:
"""
bilgi_guvenligi_qa_master.csv -> Türkçe çeviri + JSONL üretim script'i

Yapılanlar:
1) GitHub raw URL'den CSV indir
2) question & answer kolonlarını Türkçeye çevir (googletrans ile)
3) question_tr & answer_tr kolonlarını ekle ve yeni CSV kaydet
4) dataset_split'e göre:
   - bilgi_guvenligi_finetune_train_tr.jsonl
   - bilgi_guvenligi_finetune_test_tr.jsonl
   dosyalarını oluştur
"""

import pandas as pd
import json
import time
from pathlib import Path

# Eğer googletrans yoksa önce:
# pip install googletrans==4.0.0-rc1
from googletrans import Translator

# -------------------------------------------------------------------
# 1. Ayarlar
# -------------------------------------------------------------------

# Senin verdiğin URL
CSV_URL = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_qa_master.csv"

# Çıktı klasörü (istersen değiştir)
OUTPUT_DIR = Path("./bilgi_guvenligi_tr_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MASTER_TR_CSV_PATH = OUTPUT_DIR / "bilgi_guvenligi_qa_master_tr.csv"
TRAIN_JSONL_PATH   = OUTPUT_DIR / "bilgi_guvenligi_finetune_train_tr.jsonl"
TEST_JSONL_PATH    = OUTPUT_DIR / "bilgi_guvenligi_finetune_test_tr.jsonl"

# -------------------------------------------------------------------
# 2. CSV'yi indir ve oku
# -------------------------------------------------------------------

print("🔹 CSV indiriliyor ve okunuyor...")

df = pd.read_csv(CSV_URL)

print("Veri kümesi boyutu:", df.shape)
print("Kolonlar:", df.columns.tolist())
print(df.head(3))

# Beklenen önemli kolonlar:
# question, answer, question_type, is_impossible, dataset_split

required_cols = {"question", "answer", "dataset_split"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Eksik kolonlar: {missing}. CSV yapısını kontrol et.")

# -------------------------------------------------------------------
# 3. Çeviri fonksiyonu (İngilizce -> Türkçe)
# -------------------------------------------------------------------

translator = Translator()
translation_cache = {}  # Aynı cümleyi tekrar çevirmemek için

def translate_to_tr(text: str) -> str:
    """İngilizce metni Türkçeye çevirir. Basit cache ve hata yönetimi içerir."""
    if not isinstance(text, str):
        return text
    text = text.strip()
    if text == "":
        return text

    # Cache kontrolü
    if text in translation_cache:
        return translation_cache[text]

    # Google Translate isteği
    for attempt in range(3):
        try:
            result = translator.translate(text, src="en", dest="tr")
            tr_text = result.text
            translation_cache[text] = tr_text
            # Çok hızlı gitmemek için hafif bekleme
            time.sleep(0.3)
            return tr_text
        except Exception as e:
            print(f"⚠️ Çeviri hatası (deneme {attempt+1}/3): {e}")
            time.sleep(1.0)

    # 3 denemede de çevirilemediyse, orijinali döndür
    print("❌ Çevrilemeyen metin, orijinali bırakılıyor:", text[:80], "...")
    translation_cache[text] = text
    return text

# -------------------------------------------------------------------
# 4. Soru & cevap kolonlarını Türkçeye çevir ve yeni sütunlara yaz
# -------------------------------------------------------------------

print("🔹 Soru ve cevaplar Türkçeye çevriliyor... (biraz sürebilir)")

question_tr_list = []
answer_tr_list = []

for idx, row in df.iterrows():
    q_en = row["question"]
    a_en = row["answer"]
    q_tr = translate_to_tr(q_en)
    a_tr = translate_to_tr(a_en)

    question_tr_list.append(q_tr)
    answer_tr_list.append(a_tr)

    if idx % 50 == 0:
        print(f"  -> {idx} satır işlendi...")

df["question_tr"] = question_tr_list
df["answer_tr"]   = answer_tr_list

print("İlk 3 satır (TR):")
print(df[["question", "question_tr", "answer", "answer_tr"]].head(3))

# -------------------------------------------------------------------
# 5. Türkçeleştirilmiş master CSV'yi kaydet
# -------------------------------------------------------------------

df.to_csv(MASTER_TR_CSV_PATH, index=False, encoding="utf-8-sig")
print(f"✅ Türkçeleştirilmiş master CSV kaydedildi: {MASTER_TR_CSV_PATH}")

# -------------------------------------------------------------------
# 6. Train / Test split'e göre JSONL üret
# -------------------------------------------------------------------

def row_to_jsonl_line_tr(row) -> str:
    """
    Fine-tune için kullanılacak tek satırlık JSON:
    {
      "instruction": "Türkçe soru",
      "input": "",
      "output": "Türkçe cevap"
    }
    """
    record = {
        "instruction": row["question_tr"],
        "input": "",
        "output": row["answer_tr"],
    }
    # ensure_ascii=False -> Türkçe karakterler bozulmaz
    return json.dumps(record, ensure_ascii=False)

print("🔹 Train / Test JSONL dosyaları üretiliyor...")

train_df = df[df["dataset_split"] == "train"]
test_df  = df[df["dataset_split"] == "test"]

print("Train satır sayısı:", len(train_df))
print("Test satır sayısı:", len(test_df))

# TRAIN JSONL
with open(TRAIN_JSONL_PATH, "w", encoding="utf-8") as f:
    for _, row in train_df.iterrows():
        line = row_to_jsonl_line_tr(row)
        f.write(line + "\n")

# TEST JSONL
with open(TEST_JSONL_PATH, "w", encoding="utf-8") as f:
    for _, row in test_df.iterrows():
        line = row_to_jsonl_line_tr(row)
        f.write(line + "\n")

print(f"✅ Train JSONL: {TRAIN_JSONL_PATH}")
print(f"✅ Test JSONL : {TEST_JSONL_PATH}")

print("🎉 İşlem tamamlandı!")


🔹 CSV indiriliyor ve okunuyor...
Veri kümesi boyutu: (614, 9)
Kolonlar: ['question_id', 'question', 'answer', 'question_type', 'source_filenames', 'source_sections', 'source_chunk_ids', 'is_impossible', 'dataset_split']
  question_id                                           question  \
0      QA0001  What are some examples of activities that are ...   
1      QA0002  What are the potential consequences for indivi...   
2      QA0003  According to the policy, what specific securit...   

                                              answer question_type  \
0  The policy prohibits violations of intellectua...        simple   
1  Non-compliance with this policy may result in ...        simple   
2  To ensure the security of wireless networks, a...       complex   

                                    source_filenames        source_sections  \
0         SANS_Acceptable_Use_Standard_April2025.pdf             Safeguards   
1  SANS_Safeguard_Implementation_Management_Polic...             Saf

In [ ]:
from google.colab import files

files.download("bilgi_guvenligi_tr_outputs/bilgi_guvenligi_qa_master_tr.csv")
files.download("bilgi_guvenligi_tr_outputs/bilgi_guvenligi_finetune_train_tr.jsonl")
files.download("bilgi_guvenligi_tr_outputs/bilgi_guvenligi_finetune_test_tr.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>